In [ ]:
import sys,re
import numpy as np
from sklearn import linear_model
from scipy import sparse
from collections import Counter
from sklearn import grid_search
from sklearn import cross_validation

In [2]:
L2_REGULARIZATION_STRENGTH = 1e0
MINIMUM_FEAT_COUNT=5

# dictionary of feature: index
# feature = a dictionary of feature name: value
feature_vocab={}

# reverse_features[index] = feature
reverse_features=[]

# dictionary of label: index, where
# label = SHIFT, LEFTARC_DEPENDENCY_LABEL, RIGHTARC_DEPENDENCY_LABEL
label_vocab={}

# reverse_labels[index] = label
reverse_labels=[]

# number/ID of features
fmax = 0

# 1. Checking for Projectivity
“An arc from a head to a dependent is said to be projective if there is a path from the head to every word that lies between the head and the dependent in the sentence. A dependency tree is then said to be projective if all the arcs that make it up are projective.”

In [3]:
def is_connected(i, head, heads):
    prev = i
    while prev != 0:
        prev = heads[prev]
        if prev == head:
            return True
    return False

def is_projective(toks):
    """
    params: toks is a list of (idd, tok, pos, head, lab) for a sentence
    return True if and only if the sentence has a projective dependency tree
    """
    heads = {}
    for x in toks:
        idd = x[0]
        if idd not in heads:
            heads[idd] = x[3]
    for depend in heads:
        head = heads[depend]
        for i in range(1 + min(head, depend), max(head, depend)):
            if not is_connected(i, head, heads):
                return False
    return True

# 2. Creating an Oracle
Following is the implementation of the training oracle, which transforms the input dependency trees (lists of
(idd, tok, pos, head, lab) tuples) to (configuration, action) pairs in `tree_to_actions`. The helper functions `perform_shift` and `perform_arc` generate the proper configuration and action for the corresponding SHIFT, LEFTARC and RIGHTARC operations, and update the states of the stack, input buffer and arcs as needed.

In [4]:
def perform_shift(wbuffer, stack, arcs,
                  configurations, gold_transitions):
    """
    perform the SHIFT operation
    """
    # 1. append the latest configuration to configurations
    configurations.append((list(wbuffer), list(stack), list(arcs)))
    
    # 2. append the latest action to gold_transitions
    gold_transitions.append("SHIFT")
    
    # 3. update wbuffer, stack and arcs accordingly
    stack.append(wbuffer.pop())

def perform_arc(direction, dep_label,
                wbuffer, stack, arcs,
                configurations, gold_transitions):
    """
    params:
        - direction: {"LEFT", "RIGHT"}
        - dep_label: label for the dependency relations
    Perform LEFTARC_ and RIGHTARC_ operations
    """
    # 1. append the latest configuration to configurations
    configurations.append((list(wbuffer), list(stack), list(arcs)))
    
    # 2. append the latest action to gold_transitions
    gold_transitions.append("{}ARC_{}".format(direction, dep_label))
    
    # 3. update wbuffer, stack and arcs accordingly
    if direction == "RIGHT":
        arcs.append((dep_label, stack[-2], stack[-1]))
        stack.pop()
    else:
        arcs.append((dep_label, stack[-1], stack[-2]))
        stack.pop(-2)
    

def tree_to_actions(wbuffer, stack, arcs, deps):
    """
    params:
    wbuffer: a list of word indices, top of buffer is at the end of the list
    stack: a list of word indices, top of buffer is at the end of the list
    arcs: a list of (label, head, dependent) tuples

    Given wbuffer, stack, arcs and deps
    Return configurations and gold_transitions (actions)
    """
    # configurations:
    # A list of tuples of lists
    # [(wbuffer1, stack1, arcs1), (wbuffer2, stack2, arcs2), ...]
    # Keeps tracks of the states at each step
    configurations=[]

    # gold_transitions:
    # A list of action strings, e.g ["SHIFT", "LEFTARC_nsubj"]
    # Keeps tracks of the actions at each step
    gold_transitions=[]
    
    depend = {}
    while len(wbuffer) >= 0:
        if len(wbuffer) == 0 and len(stack) == 1 and stack[0] == 0: # done
            return configurations, gold_transitions
        if len(stack) < 2 and len(wbuffer) > 0: # have to shift
            perform_shift(wbuffer, stack, arcs, configurations, gold_transitions)
            continue
            
        elem1, elem2 = stack[-1], stack[-2]
        if elem1 in deps and (elem1, elem2) in deps[elem1]: # left
            perform_arc("LEFT", deps[elem1][(elem1, elem2)], wbuffer, stack, arcs, configurations, gold_transitions)
            depend[elem2] = 1

        elif elem2 in deps and (elem2, elem1) in deps[elem2]: # right, but check if assigned first
            if elem1 in deps and any([child not in depend for _, child in deps[elem1]]):
                perform_shift(wbuffer, stack, arcs, configurations, gold_transitions)
            else:
                perform_arc("RIGHT", deps[elem2][(elem2, elem1)], wbuffer, stack, arcs, configurations, gold_transitions)
                depend[elem1] = 1      

        else:
            perform_shift(wbuffer, stack, arcs, configurations, gold_transitions)

# 3. Tree Parsing with Predictions
`action_to_tree` updates the dependency tree based on the action predictions.

In [5]:
def isvalid(stack, wbuffer, action):
    """
    Helper function that returns True only if an action is
    legal given the current states of the stack and wbuffer
    """
    if action == "SHIFT" and len(wbuffer) > 0:
        return True
    if action.startswith("RIGHTARC") and len(stack) > 1 and stack[-1] != 0:
        return True
    if action.startswith("LEFTARC") and len(stack) > 1 and stack[-2] != 0:
        return True
    return False

def action_to_tree(tree, predictions, wbuffer, stack, arcs):
    """
    params:
    tree:
    a dictionary of dependency relations (head, dep_label)
        {
            child1: (head1, dep_lebel1),
            child2: (head2, dep_label2), ...
        }

    predictions:
    a numpy column vector of probabilities for different dependency labels
    as ordered by the global variable reverse_labels
    predictions.shape = (1, total number of dependency labels)

    wbuffer: a list of word indices, top of buffer is at the end of the list
    stack: a list of word indices, top of buffer is at the end of the list
    arcs: a list of (label, head, dependent) tuples
    """
    global reverse_labels
    for prob in np.argsort(-predictions, kind='quicksort')[0]:
        action = reverse_labels[prob]
        if isvalid(stack, wbuffer, action):
            if action == "SHIFT":
                stack.append(wbuffer.pop())
            elif action.startswith("LEFTARC_"):
                tree[stack[-2]] = (stack[-1], action.split("_")[1])
                stack.pop(-2)
            elif action.startswith("RIGHTARC_"):
                tree[stack[-1]] = (stack[-2], action.split("_")[1])
                stack.pop()
            return

In [6]:
def get_oracle(toks):
    """
    Return pairs of configurations + gold transitions (actions)
    from training data
    configuration = a list of tuple of:
        - buffer (top of buffer is at the end of the list)
        - stack (top of buffer is at the end of the list)
        - arcs (a list of (label, head, dependent) tuples)
    gold transitions = a list of actions, e.g. SHIFT
    """
    stack = [] # stack
    arcs = [] # existing list of arcs
    wbuffer = [] # input buffer

    # deps is a dictionary of head: dependency relations, where
    # dependency relations is a dictionary of the (head, child): label
    # deps = {head1:{
    #               (head1, child1):dependency_label1,
    #               (head1, child2):dependency_label2
    #              }
    #         head2:{
    #               (head2, child3):dependency_label3,
    #               (head2, child4):dependency_label4
    #              }
    #         }
    deps = {}

    # ROOT
    stack.append(0)

    # initialize variables
    for position in reversed(toks):
        (idd, _, _, head, lab) = position

        dep = (head, idd)
        if head not in deps:
            deps[head] = {}
        deps[head][dep] = lab

        wbuffer.append(idd)

    # configurations:
    # A list of (wbuffer, stack, arcs)
    # Keeps tracks of the states at each step
    # gold_transitions:
    # A list of action strings ["SHIFT", "LEFTARC_nsubj"]
    # Keeps tracks of the actions at each step
    configurations, gold_transitions = tree_to_actions(wbuffer, stack, arcs, deps)
    return configurations, gold_transitions

def featurize_configuration(configuration, tokens, postags):
    """
    Given configurations of the stack, input buffer and arcs,
    words of the sentence and POS tags of the words,
    return some features
    """
    wbuffer, stack, arcs = configuration
    feats = {}

    feats["%s_%s" % ("len_buffer", len(wbuffer))] = 1
    feats["%s_%s" % ("len_stack", len(stack))] = 1

    # single-word features
    if len(stack) > 0:
        feats["%s_%s" % ("stack_word_1", tokens[stack[-1]])] = 1
        feats["%s_%s" % ("stack_tag_1", postags[stack[-1]])] = 1
        feats["%s_%s_%s" % ("stack_tag_word_1", tokens[stack[-1]], postags[stack[-1]])] = 1

    if len(stack) > 1:
        feats["%s_%s" % ("stack_word_2", tokens[stack[-2]])] = 1
        feats["%s_%s" % ("stack_tag_2", postags[stack[-2]])] = 1
        feats["%s_%s_%s" % ("stack_tag_word_2", tokens[stack[-2]], postags[stack[-2]])] = 1

    if len(wbuffer) > 0:
        feats["%s_%s" % ("buffer_word_1", tokens[wbuffer[-1]])] = 1
        feats["%s_%s" % ("buffer_tag_1", postags[wbuffer[-1]])] = 1
        feats["%s_%s_%s" % ("buffer_tag_word_1", tokens[wbuffer[-1]], postags[wbuffer[-1]])] = 1

    # word-pair features
    if len(stack) > 1:
        feats["%s_%s_%s_%s" % ("stack1_word_tag_stack2_tag", tokens[stack[-1]], postags[stack[-1]], postags[stack[-2]])] = 1

    return feats

def get_oracles(filename):
    """
    Get configurations, gold_transitions from all sentences
    """
    with open(filename, encoding="utf8") as f:
        toks, tokens, postags = [], {}, {}
        tokens[0] = "ROOT"
        postags[0] = "ROOT"

        # a list of all features for each transition step
        feats = []
        # a list of labels, e.g. SHIFT, LEFTARC_DEP_LABEL, RIGHTARC_DEP_LABEL
        labels = []

        for line in f:
            cols = line.rstrip().split("\t")

            if len(cols) < 2: # at the end of each sentence
                if len(toks) > 0:
                    if is_projective(toks): # only use projective trees
                        # get all configurations and gold standard transitions
                        configurations, gold_transitions = get_oracle(toks)

                        for i in range(len(configurations)):
                            feat = featurize_configuration(configurations[i], tokens, postags)
                            label = gold_transitions[i]
                            feats.append(feat)
                            labels.append(label)

                    # reset vars for the next sentence
                    toks, tokens, postags = [], {}, {}
                    tokens[0] = "ROOT"
                    postags[0] = "ROOT"
                continue

            if cols[0].startswith("#"):
                continue

            # construct the tuple for each word in the sentence
            # for each word in the sentence
            # idd: index of a word in a sentence, starting from 1
            # tok: the word itself
            # pos: pos tag for that word
            # head: parent of the dependency
            # lab: dependency relation label
            idd, tok, pos, head, lab = int(cols[0]), cols[1], cols[4], int(cols[6]), cols[7]
            toks.append((idd, tok, pos, head, lab))

            # feature for training to predict the gold transition
            tokens[idd], postags[idd] = tok, pos

        return feats, labels

def train(feats, labels):
    """
    Train transition-based parsed to predict next action (labels)
    given current configuration (featurized by feats)
    Return the classifier trained using the logistic regression model
    """
    global feature_vocab, label_vocab, fmax, reverse_labels, reverse_features

    lid = 0 # label ID
    D = len(feats) # each row of feats corresponds to a row in labels
    feature_counts = Counter()

    # build dictionary of labels
    for i in range(D):
        for f in feats[i]:
            feature_counts[f] += 1

        if labels[i] not in label_vocab:
            label_vocab[labels[i]] = lid
            lid += 1

    # build dictionary of features
    for f in feature_counts:
        if feature_counts[f] > MINIMUM_FEAT_COUNT and f not in feature_vocab:
            feature_vocab[f] = fmax
            fmax += 1

    # build reverse lookup for features and labels
    reverse_labels = [None]*lid
    for label in label_vocab:
        reverse_labels[label_vocab[label]] = label

    reverse_features = [None]*fmax
    for feature in feature_vocab:
        reverse_features[feature_vocab[feature]] = feature

    # X is a D-by-fmax matrix, where each row represents
    # features for a configuration / actions
    # Y is a list of labels for all configurations
    X = sparse.lil_matrix((D, fmax))
    Y = []
    for i in range(D):
        for f in feats[i]:
            if f in feature_vocab:
                fid = feature_vocab[f]
                X[i,fid] = 1
        Y.append(label_vocab[labels[i]])

    print ("Docs: ", D, "Features: ", fmax)
    log_reg = linear_model.LogisticRegression(C=L2_REGULARIZATION_STRENGTH,
                                              penalty='l2')

    clf = grid_search.GridSearchCV(log_reg, {'C':(1e0,1e1)}, n_jobs=10)
    log_reg = clf.fit(sparse.coo_matrix(X), Y).best_estimator_
    print ("Best C: %s" % clf.best_estimator_)

    return log_reg

def parse(toks, logreg):
    """
    parse sentence with trained model and return correctness measure
    """
    tokens, postags = {}, {}
    tokens[0] = "ROOT"
    postags[0] = "ROOT"

    heads, labels = {}, {}

    wbuffer, stack, arcs = [], [], []
    stack.append(0)

    for position in reversed(toks):
        # featurization
        (idd, tok, pos, head, lab) = position
        tokens[idd] = tok
        postags[idd] = pos

        # keep track of gold standards for performance evaluation
        heads[idd], labels[idd] = head, lab

        # update buffer
        wbuffer.append(idd)

    tree = {}
    while len(wbuffer) >= 0:
        if len(wbuffer) == 0 and len(stack) == 0: break
        if len(wbuffer) == 0 and len(stack) == 1 and stack[0] == 0: break

        feats = (featurize_configuration((wbuffer, stack, arcs), tokens, postags))
        X = sparse.lil_matrix((1, fmax))
        for f in feats:
            if f in feature_vocab:
                X[0,feature_vocab[f]]=1

        predictions = logreg.predict_proba(X)
        # your function will be called here
        action_to_tree(tree, predictions, wbuffer, stack, arcs)

    # correct_unlabeled: total number of correct (head, child) dependencies
    # correct_labeled: total number of correctly *labeled* dependencies
    correct_unlabeled, correct_labeled, total = 0, 0, 0

    for child in tree:
        (head, label) = tree[child]
        if head == heads[child]:
            correct_unlabeled += 1
            if label == labels[child]: correct_labeled += 1
        total += 1

    return [correct_unlabeled, correct_labeled, total]

def evaluate(filename, logreg):
    """
    Evaluate the performance of a parser against gold standard
    """
    with open(filename, encoding="utf8") as f:
        toks=[]
        totals = np.zeros(3)
        for line in f:
            cols=line.rstrip().split("\t")

            if len(cols) < 2: # end of a sentence
                if len(toks) > 0:
                    if is_projective(toks):
                        tots = np.array(parse(toks, logreg))
                        totals += tots
                        #print ("%.3f\t%.3f\t%s" % (totals[0]/totals[2], totals[1]/totals[2], totals))
                    toks = []
                continue

            if cols[0].startswith("#"):
                continue

            idd, tok, pos, head, lab = int(cols[0]), cols[1], cols[4], int(cols[6]), cols[7]
            toks.append((idd, tok, pos, head, lab))
        print("%.3f\t%.3f\t%s" % (totals[0]/totals[2], totals[1]/totals[2], totals))

In [7]:
feats, labels = get_oracles("train.projective.conll")
logreg = train(feats, labels)
evaluate("dev.projective.conll", logreg)

Docs:  371858 Features:  47679


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


Best C: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.738	0.689	[17447. 16282. 23625.]
